In [ ]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import json 
from scipy.sparse import load_npz

In [ ]:
# take the labels from the y matrix, then put them into T1 csv format in the aux tasks corresponding to the main tasks
# in the next step/notebook we will join them together with the other data (melloddy/pseudolabels)

In [ ]:
# challenge is to find the corresponding auxiliary task 
# cont_classification_task_id (baseline) -> iai + threshold -> cont_classification_task_id (image_model) -> melloddy_compatible iai

In [ ]:
path = '/projects/melloddy/data/y2run_final/data_prep/y2_ext/y2_ext_noaux/results_tmp/classification/T8c.csv'
t8_baseline = pd.read_csv(path)
t8_baseline['threshold'] = t8_baseline['threshold'].round(decimals=5)

In [ ]:
path = '/projects/home/wheyndri/research/wp1_imageauxdata/15_y3_imageaux_study_taskthreshold/data/image_model_step1/image_model_step1/results_tmp/classification/T8c.csv'
t8_imagemodel = pd.read_csv(path)
t8_imagemodel['threshold'] = t8_imagemodel['threshold'].round(decimals=5)

In [ ]:
t8_mgd = pd.merge(
    t8_imagemodel
    ,t8_baseline
    ,how='inner'
    ,on=['input_assay_id','threshold']
    ,suffixes=('_image','_baseline')
)
t8_mgd = t8_mgd[~t8_mgd['cont_classification_task_id_image'].isna()]

In [ ]:
t8_mgd.head()

In [ ]:
t8_baseline.query('input_assay_id == 0')

In [ ]:
t8_imagemodel.query('cont_classification_task_id == 3')

In [ ]:
t8_imagemodel.query('input_assay_id == 0')

In [ ]:
### some auxiliary thresholds will still be added in addition to the expert thresholds 
### therefore, not all of the image model tasks correspond to a baseline task 

In [ ]:
with open('../notebooks/mapping/image_iai_cont_map.json') as fp:
    image_iai_cont_map = json.load(fp)

In [ ]:
t8_mgd.columns

In [ ]:
t8_mgd['baseline_compliant_input_assay_id_image'] = t8_mgd['cont_classification_task_id_image'].astype(int).astype(str).map(image_iai_cont_map)
t8_mgd = t8_mgd[~t8_mgd['baseline_compliant_input_assay_id_image'].isna()]
t8_mgd

In [ ]:
t8_mgd.to_csv(
    '../notebooks/mapping/baseline_image_model_task_mapping.csv'
    ,index=False
)

In [ ]:
# now filter the baseline T10c on the iais which can actually be mapped from t8_mgd

In [ ]:
t8_mgd[['input_assay_id','threshold']]

In [ ]:
## retrieve the input compound ids from T5 

In [ ]:
path = '/projects/melloddy/data/y2run_final/data_prep/y2_ext/y2_ext_noaux/results/T10c_cont.csv'
t10c_baseline = pd.read_csv(path)
path = '/projects/melloddy/data/y2run_final/data_prep/y2_ext/y2_ext_noaux/mapping_table/T5.csv'
t5_baseline = pd.read_csv(path)

In [ ]:
t10c_baseline.shape

In [ ]:
# filter on relevant tasks 

In [ ]:
t10c_baseline_lim = pd.merge(
    t10c_baseline
    ,pd.merge(
        t8_baseline
        ,t8_mgd
        ,on=['input_assay_id','threshold']
        ,how='inner'
    )
    ,on='cont_classification_task_id'
    ,how='inner'    
)

In [ ]:
t10c_baseline_lim.shape

In [ ]:
t10c_baseline_lim.head()

In [ ]:
t10c_baseline_lim2 = pd.merge(
    t5_baseline[['input_compound_id','descriptor_vector_id']].drop_duplicates()
    ,t10c_baseline_lim
    ,on='descriptor_vector_id'
    ,how='inner'
)

In [ ]:
t10c_baseline_lim2.head()

In [ ]:
t10c_baseline_lim2.shape

In [ ]:
t10c_baseline_lim2['standard_value'] = np.where(t10c_baseline_lim2['class_label'] == -1 , 0, 6)

In [ ]:
t10c_baseline_lim2['standard_qualifier'] = '='

In [ ]:
df_t1_truelabels_out = t10c_baseline_lim2[['baseline_compliant_input_assay_id_image','standard_value','standard_qualifier','input_compound_id']].drop_duplicates()

In [ ]:
df_t1_truelabels_out.to_csv('../notebooks/aux_data/T1_image_truelabels.csv', index=False)

In [ ]:
df_t1_truelabels_out.head()